<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Title" data-toc-modified-id="Title-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Title</a></span><ul class="toc-item"><li><span><a href="#Business-Understanding-and-Business-Problem" data-toc-modified-id="Business-Understanding-and-Business-Problem-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Business Understanding and Business Problem</a></span><ul class="toc-item"><li><span><a href="#Cost-of-Errors" data-toc-modified-id="Cost-of-Errors-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Cost of Errors</a></span></li></ul></li><li><span><a href="#Data-Understanding" data-toc-modified-id="Data-Understanding-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Understanding</a></span><ul class="toc-item"><li><span><a href="#Initial-Data" data-toc-modified-id="Initial-Data-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Initial Data</a></span></li></ul></li><li><span><a href="#Tensorflow-model" data-toc-modified-id="Tensorflow-model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tensorflow model</a></span></li><li><span><a href="#Exploratory-Data-Analysis" data-toc-modified-id="Exploratory-Data-Analysis-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Exploratory Data Analysis</a></span></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#Final-Model" data-toc-modified-id="Final-Model-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Final Model</a></span></li><li><span><a href="#Results/Conclusions" data-toc-modified-id="Results/Conclusions-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Results/Conclusions</a></span><ul class="toc-item"><li><span><a href="#Further-Exploration/Questions" data-toc-modified-id="Further-Exploration/Questions-1.7.1"><span class="toc-item-num">1.7.1&nbsp;&nbsp;</span>Further Exploration/Questions</a></span></li></ul></li></ul></li></ul></div>

insert header

# Title

## Business Understanding and Business Problem

### Cost of Errors

## Data Understanding

In [1]:
#imports grabbed from lecture
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

### Initial Data

In [2]:
#the four csv files provided
links_df = pd.read_csv('data/links.csv')
movies_df = pd.read_csv('data/movies.csv')
ratings_df = pd.read_csv('data/ratings.csv')
tags_df = pd.read_csv('data/tags.csv')

In [3]:
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_df.shape

(9742, 3)

In [6]:
movies_df['title'].nunique()

9737

In [7]:
movies_df['title'].value_counts().head(10)

Eros (2004)                                             2
War of the Worlds (2005)                                2
Saturn 3 (1980)                                         2
Emma (1996)                                             2
Confessions of a Dangerous Mind (2002)                  2
Nothing to Declare (Rien à déclarer) (2010)             1
Twilight Zone: The Movie (1983)                         1
Heartless (2009)                                        1
Let the Right One In (Låt den rätte komma in) (2008)    1
The Scorpion King (2002)                                1
Name: title, dtype: int64

There are 9737 unique movie titles. Repeats for 5 movies:
- Emma (1996)                                   2
- Saturn 3 (1980)                               2
- Eros (2004)                                   2
- Confessions of a Dangerous Mind (2002)        2
- War of the Worlds (2005)                      2

In [8]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [10]:
ratings_df.groupby('movieId').count()

,userId,rating,timestamp
movieId,,,
1,215,215,215
2,110,110,110
3,52,52,52
4,7,7,7
5,49,49,49
...,...,...,...
193581,1,1,1
193583,1,1,1
193585,1,1,1


In [11]:
new_ratings_df = ratings_df.drop('timestamp', axis=1).copy()

In [12]:
merged_df = new_ratings_df.merge(movies_df, how='outer', on='movieId')

In [13]:
merged_df

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100849,NaN,30892,NaN,In the Realms of the Unreal (2004),Animation|Documentary
100850,NaN,32160,NaN,Twentieth Century (1934),Comedy
100851,NaN,32371,NaN,Call Northside 777 (1948),Crime|Drama|Film-Noir
100852,NaN,34482,NaN,"Browning Version, The (1951)",Drama


In [14]:
merged_df.isna().sum()

userId     18
movieId     0
rating     18
title       0
genres      0
dtype: int64

In [15]:
merged_df.tail(20)

,userId,movieId,rating,title,genres
100834,610.0,163937,3.5,Blair Witch (2016),Horror|Thriller
100835,610.0,163981,3.5,31 (2016),Horror
100836,NaN,1076,NaN,"Innocents, The (1961)",Drama|Horror|Thriller
100837,NaN,2939,NaN,Niagara (1953),Drama|Thriller
100838,NaN,3338,NaN,For All Mankind (1989),Documentary
100839,NaN,3456,NaN,"Color of Paradise, The (Rang-e khoda) (1999)",Drama
100840,NaN,4194,NaN,I Know Where I'm Going! (1945),Drama|Romance|War
100841,NaN,5721,NaN,"Chosen, The (1981)",Drama
100842,NaN,6668,NaN,"Road Home, The (Wo de fu qin mu qin) (1999)",Drama|Romance
100843,NaN,6849,NaN,Scrooge (1970),Drama|Fantasy|Musical


In [16]:
no_nulls = merged_df.dropna()

In [17]:
no_nulls

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...
100831,610.0,160341,2.5,Bloodmoon (1997),Action|Thriller
100832,610.0,160527,4.5,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610.0,160836,3.0,Hazard (2005),Action|Drama|Thriller
100834,610.0,163937,3.5,Blair Witch (2016),Horror|Thriller


In [18]:
pivot_test = no_nulls.pivot_table(index='userId', columns='title', values='rating')

In [19]:
pivot_test.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Setting up basic model?

In [33]:
from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy

In [34]:
no_nulls_test = no_nulls.drop(['title', 'genres'], axis=1)

In [35]:
no_nulls_test.shape

(100836, 3)

In [46]:
no_nulls_test

,userId,movieId,rating
0,1.0,1,4.0
1,5.0,1,4.0
2,7.0,1,4.5
3,15.0,1,2.5
4,17.0,1,4.5
...,...,...,...
100831,610.0,160341,2.5
100832,610.0,160527,4.5
100833,610.0,160836,3.0
100834,610.0,163937,3.5


In [47]:
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
reader = Reader()
data = Dataset.load_from_df(no_nulls_test,reader)

In [48]:
type(data)

surprise.dataset.DatasetAutoFolds

In [49]:
trainset, testset = train_test_split(data, test_size=.1)

In [50]:
type(trainset)

surprise.trainset.Trainset

In [51]:
print('Number of users: ', trainset.n_users, '\n')
print('Number of items: ', trainset.n_items)

Number of users:  610 

Number of items:  9369


In [52]:
sim_cos = {'name':'msd', 'user_based':False}

In [53]:
baseline = knns.KNNBaseline()
baseline.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [54]:
baseline.sim

array([[1.        , 0.39473684, 0.3196347 , ..., 0.26315789, 0.2962963 ,
        0.625     ],
       [0.39473684, 1.        , 0.70588235, ..., 0.5       , 0.        ,
        1.        ],
       [0.3196347 , 0.70588235, 1.        , ..., 0.66666667, 0.76190476,
        0.2       ],
       ...,
       [0.26315789, 0.5       , 0.66666667, ..., 1.        , 0.        ,
        0.        ],
       [0.2962963 , 0.        , 0.76190476, ..., 0.        , 1.        ,
        0.2       ],
       [0.625     , 1.        , 0.2       , ..., 0.        , 0.2       ,
        1.        ]])

In [55]:
predictions = baseline.test(testset)

In [56]:
accuracy.rmse(predictions)

RMSE: 0.8580


0.8579947949560972

Regular KNN

In [33]:
knn_z = knns.KNNWithZScore()
knn_z.fit(trainset=trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [34]:
predictions = knn_z.test(testset)

In [35]:
accuracy.rmse(predictions)

RMSE: 0.8890


0.8890124431796815

In [36]:
from surprise.prediction_algorithms import BaselineOnly

In [39]:
baseline = BaselineOnly()
baseline.fit(trainset)

Estimating biases using als...


In [42]:
baseline.default_prediction()

3.501214855952794

In [43]:
predictions = baseline.test(testset)

In [44]:
accuracy.rmse(predictions)

RMSE: 0.8704


0.870410990948204

## Tensorflow model

This is hard. Gonna put it on the back burner

In [44]:
# from keras.layers import Embedding, Dot, Input
# from keras.models import Model
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split

In [28]:
no_nulls.head(5)

,userId,movieId,rating,title,genres
0,1.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5.0,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17.0,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [41]:
# setting x and y vars
X = no_nulls[['userId', 'movieId']]
y = no_nulls['rating']

In [36]:
# scaling y for ease of processing 
y = np.array(y).reshape(-1, 1)
scaler = MinMaxScaler().fit(y)
y_scaled = scaler.transform(y)

In [40]:
y_scaled.shape

(100836, 1)

In [43]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, random_state=42)

In [ ]:
# creating a model
model = R

In [22]:
# Input layers for user and item
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

In [23]:
user_input

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'input_1')>

In [24]:
user_embedding = Embedding(num_users, latent_dim, input_length=1)(user_input)
item_embedding = Embedding(num_items, latent_dim, input_length=1)(item_input)

NameError: name 'num_users' is not defined

## Exploratory Data Analysis

## Modeling

## Final Model

## Results/Conclusions

### Further Exploration/Questions